In [9]:
import pandas as pd

df = pd.read_csv("data/2022_place_canvas_history-000000000000.csv.gzip", parse_dates=[0], compression="gzip")

df.head()

,timestamp,user_id,pixel_color,coordinate
0,2022-04-04 00:53:51.577000+00:00,ovTZk4GyTS1mDQnTbV+vDOCu1f+u6w+CkIZ6445vD4XN8a...,#00CCC0,"826,1048"
1,2022-04-04 00:53:53.758000+00:00,6NSgFa1CvIPly1VniNhlbrmoN3vgDFbMSKqh+c4TTfrr3d...,#94B3FF,"583,1031"
2,2022-04-04 00:53:54.685000+00:00,O5Oityp3Z3owzTuwM9XnMggpLcqKEumsOMKGhRiDTTImWb...,#6A5CFF,"1873,558"
3,2022-04-04 00:54:57.541000+00:00,tc273UiqS0wKa6VwiOs/iz/t4LyPYrhL2Q347awn11IQQE...,#009EAA,"1627,255"
4,2022-04-04 00:55:16.307000+00:00,OOWsU/HLb4UUkQwclDeXFtsJTOXMlAdNHiRpFA1Qk+SxUr...,#94B3FF,"49,1478"


In [12]:
df['user_id'] = df['user_id'].astype('string')
df['pixel_color'] = df['pixel_color'].astype('string')

df["coordinate"] = df["coordinate"].apply(lambda x: float(x.replace(",", ".")) if isinstance(x, str) else float(x))

In [13]:
df.dtypes

timestamp      datetime64[ns, UTC]
user_id                     string
pixel_color                 string
coordinate                 float64
dtype: object